<a href="https://colab.research.google.com/github/Abinaya-J/Udacity_SQL_for_Data_Analysis/blob/master/Lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Subqueries
#We want to find the average number of events for each day for each channel. The first table will provide us the number of events for each day and channel, 
#and then we will need to average these values together using a second query.
SELECT channel,AVG(event_count) avg_event_count
FROM (
SELECT DATE_TRUNC('day',occurred_at) day_occured,channel,COUNT(*) event_count
FROM web_events
GROUP BY 1,2
) sub
GROUP BY 1

In [ ]:
#Pull the first month/year combo from the orders table
#Find orders that took place in the same year/month combo and the average quantities of each type of paper
SELECT AVG(standard_qty) avg_std, AVG(gloss_qty) avg_gls, AVG(poster_qty) avg_pst,SUM(total_amt_usd)
FROM orders
WHERE DATE_TRUNC('month',occurred_at)= 
     (SELECT DATE_TRUNC('month',MIN(occurred_at)) month_occured
FROM orders)

In [ ]:
#More on Subqueries
#Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
SELECT sub3.rep_name,sub3.region_name,sub3.total_amt
FROM(SELECT region_name,MAX(total_amt) total_max_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
          FROM sales_reps s
          JOIN accounts a
          ON a.sales_rep_id = s.id
          JOIN orders o
          ON o.account_id = a.id
          JOIN region r
          ON r.id = s.region_id
          GROUP BY 1, 2) sub1                         
          GROUP BY 1)
JOIN(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1, 2
     ORDER BY 3 DESC)sub3
ON sub3.region_name = sub2.region_name AND sub3.total_amt = sub2.total_max_amt;

#For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?
SELECT r.name,COUNT(o.total) total_orders
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name
HAVING SUM(o.total_amt_usd)=(SELECT MAX(total_amt)
                            FROM(SELECT r.name region_name, SUM(o.total_amt_usd) total_amt
                                FROM sales_reps s
                                JOIN accounts a
                                ON a.sales_rep_id = s.id
                                JOIN orders o
                                ON o.account_id = a.id
                                JOIN region r
                                ON r.id = s.region_id
                                GROUP BY 1) );

#How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?
SELECT COUNT(*) 
FROM (SELECT a.name
FROM accounts a
JOIN orders o
ON a.id= o.account_id
GROUP BY 1
HAVING SUM(o.total) >(SELECT total_ord
                     FROM(SELECT a.name, SUM(o.standard_qty) tot_std_qty, SUM(o.total) total_ord
                          FROM accounts a
                          JOIN orders o
                          ON a.id= o.account_id
                          GROUP BY 1
                          ORDER BY 2 DESC
                          lIMIT 1)));

#For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?
SELECT a.id,w.channel,COUNT(*)
FROM a
JOIN web_events w
ON a.id=w.account_id
WHERE a.id=(SELECT a.id
FROM(SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
    FROM orders o
    JOIN accounts a
    ON a.id = o.account_id
    GROUP BY 1,2
    ORDER BY 3 DESC
    LIMIT 1))
GROUP BY 1,2
ORDER BY 3 DESC;

#What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?
SELECT AVG(tot_spent)
FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
      FROM orders o
      JOIN accounts a
      ON a.id = o.account_id
      GROUP BY a.id, a.name
      ORDER BY 3 DESC
      LIMIT 10)

#What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, 
#on average, than the average of all orders.
SELECT AVG(avg_amt)
FROM (SELECT o.account_id, AVG(o.total_amt_usd) avg_amt
      FROM orders o
      GROUP BY 1
      HAVING AVG(o.total_amt_usd) > (SELECT AVG(o.total_amt_usd) avg_all
                                   FROM orders o));

In [ ]:
#WITH Quizzes
#Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
WITH t1 AS (
  SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
   FROM sales_reps s
   JOIN accounts a
   ON a.sales_rep_id = s.id
   JOIN orders o
   ON o.account_id = a.id
   JOIN region r
   ON r.id = s.region_id
   GROUP BY 1,2
   ORDER BY 3 DESC), 
t2 AS (
   SELECT region_name, MAX(total_amt) total_amt
   FROM t1
   GROUP BY 1)
SELECT t1.rep_name, t1.region_name, t1.total_amt
FROM t1
JOIN t2
ON t1.region_name = t2.region_name AND t1.total_amt = t2.total_amt;

#For the region with the largest sales total_amt_usd, how many total orders were placed?
WITH t1 AS (
   SELECT r.name region_name, SUM(o.total_amt_usd) total_amt
   FROM sales_reps s
   JOIN accounts a
   ON a.sales_rep_id = s.id
   JOIN orders o
   ON o.account_id = a.id
   JOIN region r
   ON r.id = s.region_id
   GROUP BY r.name), 
t2 AS (
   SELECT MAX(total_amt)
   FROM t1)
SELECT r.name, COUNT(o.total) total_orders
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name
HAVING SUM(o.total_amt_usd) = (SELECT * FROM t2);